<a href="https://colab.research.google.com/github/MWFK/Ensemble-Learning-Zero-to-Hero/blob/main/1_Randomized_Search_vs_Grid_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Comparing randomized search and grid search for hyperparameter estimation


Compare randomized search and grid search for optimizing hyperparameters of a
linear SVM with SGD training.
All parameters that influence the learning are searched simultaneously
(except for the number of estimators, which poses a time / quality tradeoff).

The randomized search and the grid search explore exactly the same space of
parameters. The result in parameter settings is quite similar, while the run
time for randomized search is drastically lower.

The performance is may slightly worse for the randomized search, and is likely
due to a noise effect and would not carry over to a held-out test set.

Note that in practice, one would not search over this many different parameters
simultaneously using grid search, but pick only the ones deemed most important.

https://scikit-learn.org/stable/auto_examples/model_selection/plot_randomized_search.html


In [1]:
%matplotlib inline

# GridSearchCV

In [3]:
#print(__doc__)
import numpy as np
from time import time
import scipy.stats as stats
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import load_digits
from sklearn.linear_model import SGDClassifier

# get some data
X, y = load_digits(return_X_y=True)

# build a classifier
clf = SGDClassifier(loss='hinge', penalty='elasticnet',fit_intercept=True)

# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})"
                  .format(results['mean_test_score'][candidate],
                          results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

# use a full grid over all parameters
param_grid = {'average': [True, False],
              'l1_ratio': np.linspace(0, 1, num=10),
              'alpha': np.power(10, np.arange(-4, 1, dtype=float))}

# run grid search
grid_search = GridSearchCV(clf, param_grid=param_grid)
start = time()
grid_search.fit(X, y)

print("GridSearchCV took %.2f seconds for %d candidate parameter settings."
      % (time() - start, len(grid_search.cv_results_['params'])))
report(grid_search.cv_results_)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_mo

GridSearchCV took 135.60 seconds for 100 candidate parameter settings.
Model with rank: 1
Mean validation score: 0.929 (std: 0.026)
Parameters: {'alpha': 0.0001, 'average': True, 'l1_ratio': 0.0}

Model with rank: 2
Mean validation score: 0.928 (std: 0.029)
Parameters: {'alpha': 0.0001, 'average': True, 'l1_ratio': 0.4444444444444444}

Model with rank: 3
Mean validation score: 0.926 (std: 0.035)
Parameters: {'alpha': 0.01, 'average': False, 'l1_ratio': 0.3333333333333333}



# RandomizedSearchCV

In [4]:
print(__doc__)

import numpy as np

from time import time
import scipy.stats as stats
#from sklearn.utils.fixes import loguniform # import bug
#from loguniform import LogUniform # rvs() random_state error # this lib mimics the scipy.stats and it's implemnetd in scypi.stats.reciprocal
from scipy.stats import reciprocal # this replace the last tow previous libs
# loguniform/reciprocal : the loguniform is an approximate distribution, usually used in expert estimates to desccribe a variable that might take w wide range



from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.datasets import load_digits
from sklearn.linear_model import SGDClassifier

# get some data
X, y = load_digits(return_X_y=True)

# build a classifier
clf = SGDClassifier(loss='hinge', penalty='elasticnet',
                    fit_intercept=True)

# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})"
                  .format(results['mean_test_score'][candidate],
                          results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

# specify parameters and distributions to sample from
param_dist = {'average': [True, False],
              'l1_ratio': stats.uniform(0, 1),
              'alpha': reciprocal(1e-4, 1e0)}

# run randomized search
n_iter_search = 20
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search)

start = time()
random_search.fit(X, y)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))
report(random_search.cv_results_)

Automatically created module for IPython interactive environment
RandomizedSearchCV took 24.43 seconds for 20 candidates parameter settings.
Model with rank: 1
Mean validation score: 0.927 (std: 0.034)
Parameters: {'alpha': 0.00018876332719840208, 'average': True, 'l1_ratio': 0.008115873004421403}

Model with rank: 2
Mean validation score: 0.922 (std: 0.030)
Parameters: {'alpha': 0.0011589403951646953, 'average': True, 'l1_ratio': 0.30034396045842104}

Model with rank: 3
Mean validation score: 0.917 (std: 0.026)
Parameters: {'alpha': 0.36950301734634594, 'average': False, 'l1_ratio': 0.14418787298583324}

